This is a simple baseline for NER detection for Coleridge competition . Here the idea is to train a simple Spacy phrase matcher to identify the labels which we encounter in the train set to test set. This can also generalize into a very strong final solution if you are looking at small subset of dataset to recognize in your data. 

This notebook scores 0.48 in F1 metric in private test set. 

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import json
import re
import spacy
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English

In [ ]:
df=pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 150000000

terms=df['dataset_label'].unique().tolist()

matcher = PhraseMatcher(nlp.vocab)

# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add('Dataset_Label',patterns)

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [ ]:
test_root='../input/coleridgeinitiative-show-us-the-data/test'
test_files=os.listdir('../input/coleridgeinitiative-show-us-the-data/test')

In [ ]:
TEST_DATA = dict()

for file in test_files:
    with open(os.path.join(test_root,file), encoding="utf8") as f:
        TEXTS = json.loads(f.read())

        
    file_list=[]
        
    for i,(key,value) in enumerate(TEXTS):
        file_list.append(TEXTS[i][value])
    
    TEST_DATA[file.split('.')[0]]=file_list

In [ ]:
test_labels=dict()

for key,item in TEST_DATA.items():
    labels=[]
    for i in np.arange(len(item)):
        
        if (len(TEST_DATA[key][i])<100000):
            doc=nlp(TEST_DATA[key][i],disable=["tagger", "parser",'ner','textcat','lemmatizer'])
            matches = matcher(doc)
            for match_id, start, end in matches:
                span = doc[start:end]  # The matched span
                labels.append(clean_text(span.text))
  
            
    labels=set(labels)
    test_labels[key]="|".join(str(e) for e in labels)
        
df = pd.DataFrame(list(test_labels.items()),columns = ['Id','PredictionString']) 
    
df['PredictionString'].fillna('alzheimer s disease neuroimaging initiative',inplace=True)

df.to_csv('./submission.csv',index=False)



In [ ]:
df.head()